This is a youtube video from one of the popular channel 'Veritasium' that explains why most people fail in their New Year Resolution. Then, he offers research backed suggestions on how to Succeed.

In [2]:
!pip install -qqq git+https://github.com/openai/whisper.git #install whisper getting from github
!sudo apt update && sudo apt install -qqq ffmpeg #ffmpeg allows us to work with audio and video files
!pip install -qqq pytube

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 58.6 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [55.4 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [927 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,092 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/

## Generate Text from yt video

In [3]:
from pytube import YouTube

def download_youtube_audio(video_url):
    yt = YouTube(video_url)
    stream = yt.streams.filter(only_audio=True).first()
    audio_file_path = yt.title + ".mp3"  # Output audio file path

    stream.download(filename=audio_file_path)

    return audio_file_path

In [4]:
# Main code flow
video_url = "https://www.youtube.com/watch?v=Pm9CQn07OjU"
audio_file_path = download_youtube_audio(video_url)
print("Audio downloaded and saved as:", audio_file_path)

Audio downloaded and saved as: Why New Years Resolutions Fail & How To Succeed.mp3


In [5]:
 # run whisper asr
#!whisper "/content/Why New Years Resolutions Fail & How To Succeed.mp3" --model medium.en
!whisper "/content/Why New Years Resolutions Fail & How To Succeed.mp3" --model medium.en --output_format txt

100%|█████████████████████████████████████| 1.42G/1.42G [01:44<00:00, 14.6MiB/s]
[00:00.000 --> 00:02.960]  Most New Year's resolutions fail.
[00:02.960 --> 00:06.560]  So in this video, I want to talk about the science of why they fail
[00:06.560 --> 00:10.680]  and how to avoid that so your New Year's resolutions actually succeed.
[00:10.680 --> 00:15.840]  And I want to tell you about three of my New Year's resolutions for 2020.
[00:15.840 --> 00:19.040]  The first one is to stop going to news websites.
[00:19.040 --> 00:22.720]  I find I kill a ton of time by doing that.
[00:22.720 --> 00:26.680]  Now don't get me wrong, I do think it's important to know what's going on in the world,
[00:26.720 --> 00:31.680]  but I just don't think following the day-to-day developments of the news cycle is the way to do that.
[00:31.680 --> 00:38.400]  So instead, I'm going to get one newspaper delivered to my house every week.
[00:38.400 --> 00:43.160]  And that is how I am going to stay informed

## Generate Q/A

Now, Ask for 5 Q/A on the text generated from the video.
I am using langchain for OpenAI's chatCompletion API.

In [30]:
!pip install -qqq openai
import openai

client = openai.OpenAI()

def get_completion(prompt, model="gpt-3.5-turbo"):
  messages = [{"role": "user", "content": prompt}]
  response = client.chat.completions.create(
      model=model,
      messages=messages,
      temperature=0.3,
  )
  return response.choices[0].message.content

In [22]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/drive/')
!pip install -qqq -r  "/content/drive/MyDrive/LLMProjects/requirements.txt"
import os

# Read and set the environment variable from the .bashrc file
with open('/content/drive/MyDrive/LLMProjects/.bashrc') as file:
    for line in file:
        if line.startswith('export '):
            var, value = line[len('export '):].strip().split('=')
            os.environ[var] = value

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Load the text file from video

In [23]:
# Load the text from the file
with open('Why New Years Resolutions Fail & How To Succeed.txt', 'r') as file:
    transcript_text = file.read()

In [31]:
# Define a prompt to generate the Q/A pairs

prompt = (
    "Your are an AI that generates insightful questions and their answers from a given text.\n"
    "Here is the text: \n\n"
    f"{transcript_text}\n\n"
    "Generate 5 question-answer pairs related to the topic discussed in the text.\n"
)

# Make a request to the OpenAI API
response = get_completion(prompt)

qa_text = response
print(qa_text)

1. Question: Why do most New Year's resolutions fail?
Answer: Most New Year's resolutions fail because people often set overly ambitious goals that are difficult to maintain over time.

2. Question: How can setting small, specific goals help with achieving New Year's resolutions?
Answer: Setting small, specific goals can help with achieving New Year's resolutions because they are more manageable and easier to stick to consistently, leading to long-term success.

3. Question: What is the importance of writing down goals when it comes to achieving them?
Answer: Writing down goals increases the likelihood of achieving them by 40%, as it helps to clarify and track progress, providing a sense of accountability and motivation.

4. Question: Why is relying solely on willpower not an effective strategy for sticking to New Year's resolutions?
Answer: Willpower is a finite resource that can be depleted, making it difficult to resist temptation and maintain motivation over time. It is more effect

## Convert Q/A into Audio
This is done using Google Text-to-Speech 'gtts' and python's 'pydub' libraries

Credit to ChatGPT for recommending these libraries including the following code for audio generation.

In [33]:
!pip install -qqq gtts pydub
!apt-get install -qqq ffmpeg


In [34]:
from gtts import gTTS
from pydub import AudioSegment
import os

# Split Q/A into questions and answers
qa_pairs = qa_text.strip().split('\n\n')

# Create audio files for each Q/A pair
question_audio = []
answer_audio = []

for pair in qa_pairs:
    q, a = pair.split('\n')

    # Convert question to audio
    tts_question = gTTS(q, lang='en')
    question_file = 'question.mp3'
    tts_question.save(question_file)
    question_audio.append(AudioSegment.from_mp3(question_file))

    # Convert answer to audio
    tts_answer = gTTS(a, lang='en')
    answer_file = 'answer.mp3'
    tts_answer.save(answer_file)
    answer_audio.append(AudioSegment.from_mp3(answer_file))

    # Remove temporary files
    os.remove(question_file)
    os.remove(answer_file)

# Concatenate all questions and answers into one audio file
combined_audio = AudioSegment.empty()
for q_audio, a_audio in zip(question_audio, answer_audio):
    combined_audio += q_audio + AudioSegment.silent(duration=500) + a_audio + AudioSegment.silent(duration=1000)

# Save the final combined audio file
combined_audio.export('qa_audio.mp3', format='mp3')

# Play the audio file
from IPython.display import Audio
Audio('qa_audio.mp3')